In [1]:
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas_ta as ta;
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot
import difflib
import yfinance as yf
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM
from itertools import islice
import itertools
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import mplfinance as mpf
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
import pickle
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics


from sklearn.utils import class_weight


def add_trend_column(df, n_mv, n_backtrack):
    ma = df[f'MA{n_mv}']
    trend = []

    for i, _ in df.iterrows():
        if i < n_mv + n_backtrack - 1:
            trend.append(np.nan)  # Handle NaN values at the beginning or in MA{n_mv}
        elif len(ma[i-n_backtrack:i]) >= n_backtrack and all(np.sort(ma[i-n_backtrack:i]) < np.sort(ma[i-(n_backtrack-1):i+1])) and ma[i-1] < ma[i]:
            trend.append(1)
        elif len(ma[i-n_backtrack:i]) >= n_backtrack and all(np.sort(ma[i-n_backtrack:i]) > np.sort(ma[i-(n_backtrack-1):i+1])) and ma[i-1] > ma[i]:
            trend.append(-1)
        else:
            trend.append(0)

    df[f'trend_ma{n_mv}_backtrack{n_backtrack}'] = trend
    return df

def add_crp_column(df, trend_col):
    crp = [np.nan] * len(df)  # initialize with NaN for the first row
    for i, _ in df.iterrows():
        if pd.isna(df.loc[i, trend_col]):
            crp[i] = np.nan
        elif df[trend_col].iloc[i] == 1 and df['Close'].iloc[i] <= df['Close'].iloc[i-1]:
            crp[i] = 1
        elif df[trend_col].iloc[i] == -1 and df['Close'].iloc[i] >= df['Close'].iloc[i-1]:
            crp[i] = -1
        else:
            crp[i] = 0
    df[f'CRP_{trend_col}'] = crp
    return df

def add_ctp_column(df, trend_col):
    ctp = [np.nan] * len(df)  # initialize with NaN for the first row
    for i, _ in df.iterrows():
        if pd.isna(df.loc[i, trend_col]):
            ctp[i] = np.nan
        elif df[trend_col].iloc[i] == 1 and df['Close'].iloc[i] >= df['Close'].iloc[i-1]:
            ctp[i] = 1
        elif df[trend_col].iloc[i] == -1 and df['Close'].iloc[i] <= df['Close'].iloc[i-1]:
            ctp[i] = -1
        else:
            ctp[i] = 0
    df[f'CTP_{trend_col}'] = ctp
    return df

def add_rp_column(df, ma_col, crp_col, alpha, theta):
    crp = df[crp_col]
    ma5 = df[ma_col]
    rp = [np.nan] * len(df)
    for i, _ in df.iterrows():
        if (np.isnan(crp.iloc[i])) | (i + alpha >= len(df)):
            rp[i] = np.nan
        elif crp.iloc[i] == 1 and ((ma5.iloc[i] - ma5.iloc[i+1:i+alpha+1].min()) / ma5.iloc[i]) > theta/100:
            rp[i] = 2
        elif crp.iloc[i] == -1 and ((ma5.iloc[i+1:i+alpha+1].max() - ma5.iloc[i]) / ma5.iloc[i]) > theta/100:
            rp[i] = 1
        else:
            rp[i] = 0
    df[f'RP_{ma_col}_{crp_col}_alpha{alpha}_theta{theta}'] = rp
    return df


def add_tp_column(df, ma_col, ctp_col, alpha, theta):
    ctp = df[ctp_col]
    ma5 = df[ma_col]
    tp = [np.nan] * len(df)
    for i, _ in df.iterrows():
        if (np.isnan(ctp.iloc[i])) | (i + alpha >= len(df)):
            tp[i] = np.nan
        elif ctp.iloc[i] == -1 and ((ma5.iloc[i] - ma5.iloc[i+1:i+alpha+1].min()) / ma5.iloc[i]) > theta/100:
            tp[i] = 2
        elif ctp.iloc[i] == 1 and ((ma5.iloc[i+1:i+alpha+1].max() - ma5.iloc[i]) / ma5.iloc[i]) > theta/100:
            tp[i] = 1
        else:
            tp[i] = 0
    df[f'TP_{ma_col}_{ctp_col}_alpha{alpha}_theta{theta}'] = tp
    return df


def add_vr(df, window):
    df['Volume_A_temp'] = df.apply(lambda row: row['Volume'] if row['Close'] > row['Open'] else 0, axis=1)
    df['Volume_D_temp'] = df.apply(lambda row: row['Volume'] if row['Close'] < row['Open'] else 0, axis=1)
    df['Volume_U_temp'] = df.apply(lambda row: row['Volume'] if row['Close'] == row['Open'] else 0, axis=1)
    df['Volume_A'] = df['Volume_A_temp'].rolling(window).sum()
    df['Volume_D'] = df['Volume_D_temp'].rolling(window).sum()
    df['Volume_U'] = df['Volume_U_temp'].rolling(window).sum()
    df[f'vr_{window}'] = df.apply(lambda row: (row['Volume_A'] + row['Volume_U']/2) / (row['Volume_D'] + row['Volume_U']/2) * 100 if row['Volume_D'] + row['Volume_U'] > 0 else row['Volume_A'], axis=1)
    df.drop(['Volume_A_temp', 'Volume_D_temp', 'Volume_U_temp', 'Volume_A', 'Volume_D', 'Volume_U'], axis=1, inplace=True)
    return df


def add_pl(df, window):
    df['If_A'] = df.apply(lambda row: 1 if row['Close'] > row['Open'] else 0, axis=1)
    df['n_A'] = df['If_A'].rolling(window).sum()   
    df[f'pl_{window}'] = df['n_A']/window
    df.drop(['If_A','n_A'], axis=1, inplace=True)
    return df

def create_data(stock_list, require_label=True):
    df_list = []
    for s in stock_list:

        #READ DF
        ticker = yf.Ticker(s)
        df = ticker.history(period='max')
        df.index = df.index.tz_localize(None)
        df = df.loc[df.index >= '1997']
        df = df.reset_index()

        # #CREATE MV
        df['MA3'] = df.Close.rolling(3).mean()
        df['MA5'] = df.Close.rolling(5).mean()
        df['MA10'] = df.Close.rolling(10).mean()
        df['MA20'] = df.Close.rolling(20).mean()
        df['MA50'] = df.Close.rolling(50).mean()
        df['MA200'] = df.Close.rolling(200).mean()
        df['EMA5'] = ta.ema(df['Close'], 5)
        df['EMA10'] = ta.ema(df['Close'], 10)
        df['EMA20'] = ta.ema(df['Close'], 20)
        df['EMA50'] = ta.ema(df['Close'], 50)
        df['EMA200'] = ta.ema(df['Close'], 200)

        #CREATE LABEL
        df = add_trend_column(df,3,6)
        df = add_trend_column(df,5,6)
        df = add_crp_column(df,"trend_ma3_backtrack6")
        df = add_crp_column(df,"trend_ma5_backtrack6")
        df = add_ctp_column(df,"trend_ma3_backtrack6")
        df = add_ctp_column(df,"trend_ma5_backtrack6")
        if require_label == True:
            df = add_rp_column(df,'MA5','CRP_trend_ma5_backtrack6',alpha=10, theta=3)
            df = add_rp_column(df,'MA3','CRP_trend_ma3_backtrack6',alpha=10, theta=3)
            df = add_rp_column(df,'MA5','CRP_trend_ma5_backtrack6',alpha=5, theta=5)
            df = add_rp_column(df,'MA3','CRP_trend_ma3_backtrack6',alpha=5, theta=5)
            df = add_tp_column(df,'MA5','CTP_trend_ma5_backtrack6',alpha=10, theta=3)
            df = add_tp_column(df,'MA3','CTP_trend_ma3_backtrack6',alpha=10, theta=3)
            df = add_tp_column(df,'MA5','CTP_trend_ma5_backtrack6',alpha=5, theta=5)
            df = add_tp_column(df,'MA3','CTP_trend_ma3_backtrack6',alpha=5, theta=5)

        else:
            pass

        #CREATE FEATURES
        df['candle'] = np.where(df['Close'] > df['Open'], 1, np.where(df['Close'] < df['Open'], -1, 0))
        df['body'] = abs(df['Open'] - df['Close']) / df['Close'].shift(1)
        df['toptail'] = (df['High'] - df[['Open', 'Close']].max(axis=1)) / df['Close'].shift(1)
        df['bottomtail'] = (df[['Open', 'Close']].min(axis=1) - df['Low']) / df['Close'].shift(1)
        df['whole'] = (df['High'] - df['Low']) / df['Close'].shift(1)
        
        df['emacd_5_10'] = df['EMA5'] - df['EMA10']
        df['emacd_5_20'] = df['EMA5'] - df['EMA20']
        df['emacd_10_20'] = df['EMA10'] - df['EMA20']
        df['emacd_50_200'] = df['EMA50'] - df['EMA200']
        df['macd_5_10'] = df['MA5'] - df['MA10']
        df['macd_5_20'] = df['MA5'] - df['MA20']
        df['macd_10_20'] = df['MA10'] - df['MA20']   
        df['macd_50_200'] = df['MA50'] - df['MA200']
        
        df['roc_5'] = (df['Close'] - df['Close'].shift(5-1)) / df['Close'].shift(5-1) * 100
        df['rocma_5'] = (df['MA5'] - df['MA5'].shift(5-1)) / df['MA5'].shift(5-1) * 100
        df['rocema_5'] = (df['EMA5'] - df['EMA5'].shift(5-1)) / df['EMA5'].shift(5-1) * 100
        df['roc_10'] = (df['Close'] - df['Close'].shift(10-1)) / df['Close'].shift(10-1) * 100
        df['rocma_10'] = (df['MA10'] - df['MA10'].shift(10-1)) / df['MA10'].shift(10-1) * 100
        df['rocema_10'] = (df['EMA10'] - df['EMA10'].shift(10-1)) / df['EMA10'].shift(10-1) * 100
        df['roc_20'] = (df['Close'] - df['Close'].shift(20-1)) / df['Close'].shift(20-1) * 100
        df['rocma_20'] = (df['MA20'] - df['MA20'].shift(20-1)) / df['MA20'].shift(20-1) * 100
        df['rocema_20'] = (df['EMA20'] - df['EMA20'].shift(20-1)) / df['EMA20'].shift(20-1) * 100
       
        df['stok_5'] = (df['Close'] - df['Close'].rolling(window=5).min())/(df['Close'].rolling(window=5).max() - 
                                                                            df['Close'].rolling(window=5).min()) * 100
        df['stok_10'] = (df['Close'] - df['Close'].rolling(window=10).min())/(df['Close'].rolling(window=10).max() - 
                                                                            df['Close'].rolling(window=10).min()) * 100
        df['stok_20'] = (df['Close'] - df['Close'].rolling(window=20).min())/(df['Close'].rolling(window=20).max() - 
                                                                            df['Close'].rolling(window=20).min()) * 100
        df['stod_5'] = df['stok_5'].rolling(window=3).mean()
        df['stod_10'] = df['stok_10'].rolling(window=3).mean()
        df['stod_20'] = df['stok_20'].rolling(window=3).mean()
        df['stor_5'] = (df['Close'].rolling(window=5).max()-df['Close'])/(df['Close'].rolling(window=5).max()- 
                                                                           df['Close'].rolling(window=5).min()) * 100
        df['stor_10'] = (df['Close'].rolling(window=10).max()-df['Close'])/(df['Close'].rolling(window=10).max()- 
                                                                           df['Close'].rolling(window=10).min()) * 100
        df['stor_20'] = (df['Close'].rolling(window=20).max()-df['Close'])/(df['Close'].rolling(window=20).max()- 
                                                                           df['Close'].rolling(window=20).min()) * 100
        
        
        df['m'] = (df['High'] + df['Low'] + df['Close'])/3
        df['cci_14'] = (df['m'] - df['m'].rolling(window=14).mean()) / (0.015 * df['m'].rolling(window=14).std())
        df['ccis_14'] = df['cci_14'].rolling(window=9).mean()
       
        df['rsi_14'] = 100-100/(1 + (df['Close'].diff().apply(lambda x: np.where(x > 0, x, 0)).rolling(window=14).mean()/
                                     abs(df['Close'].diff().apply(lambda x: np.where(x < 0, x, 0)).rolling(window=14).mean())))
        df = add_vr(df, 20)
        
        df = add_pl(df, 20)
        
        df['vma_20'] = df['Volume'].rolling(window=20).mean()
        df['pctmv_20'] = df['Volume']/df['vma_20'].shift(1)
     
        df['aratio_20'] =  (df['High'].rolling(window=20).sum() + df['Open'].rolling(window=20).sum()) / (df['Open'].rolling(window=20).sum() + df['Low'].rolling(window=20).sum()) * 100
        df['bratio_20'] =  (df['High'].rolling(window=20).sum() + df['Close'].rolling(window=20).sum()) / (df['Close'].rolling(window=20).sum() + df['Low'].rolling(window=20).sum()) * 100
        df['abratio_20'] = df['aratio_20'] / df['bratio_20'] 
      
        df['pctchange'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)

        df = df.dropna()
        
        #ADD STOCK NAME
        df['stock'] = s 

        #DONE PROCESSING
        df_list.append(df)
        
    return df_list



def create_sliding_window_data(df_list, window_size,output_col):

    all_stock_X = np.empty([0, window_size, len(input_cols)])
    all_stock_y = np.empty([0,1])

    for df in df_list:
        X_data = df[input_cols].values
        y_data = df[output_col].values
        y_data = np.reshape(y_data, (-1, 1))
        X = []
        y = []  
        scaler = StandardScaler()
        for i in range(len(X_data)+1):
            if i < window_size:
                pass
            else:
                X_data_scaled = scaler.fit_transform(X_data[i-window_size:i])
                X.append(X_data_scaled)
                y.append(y_data[i-1])
        X = np.array(X)
        y = np.array(y)   

        all_stock_X = np.concatenate((all_stock_X, X), axis=0)
        all_stock_y = np.concatenate((all_stock_y, y), axis=0)


    return all_stock_X, all_stock_y

                               
def re_balance(X, y):
    class_counts = np.bincount(y.flatten())
    min_count = np.min(class_counts)

    class_labels = np.unique(y)

    sample_indices = []
    for label in class_labels:
        indices = np.where(y.flatten() == label)[0]
        sample_indices.extend(np.random.choice(indices, min_count, replace=False))

    np.random.shuffle(sample_indices)

    X_balanced = X[sample_indices]
    y_balanced = y[sample_indices]

    return X_balanced, y_balanced


input_cols = ['trend_ma3_backtrack6',
       'trend_ma5_backtrack6', 'CRP_trend_ma3_backtrack6',
       'CRP_trend_ma5_backtrack6','candle','body', 'toptail',
       'bottomtail', 'whole', 'emacd_5_10', 'emacd_5_20', 'emacd_10_20',
       'macd_5_10', 'macd_5_20', 'macd_10_20', 'roc_5', 'rocma_5', 'rocema_5',
       'roc_10', 'rocma_10', 'rocema_10', 'roc_20', 'rocma_20', 'rocema_20',
       'stok_5', 'stok_10', 'stok_20', 'stod_5', 'stod_10', 'stod_20',
       'stor_5', 'stor_10', 'stor_20', 'm', 'cci_14', 'ccis_14', 'rsi_14', 'vr_20', 'pl_20',
       'vma_20', 'pctmv_20', 'aratio_20', 'bratio_20','abratio_20','pctchange']



# # df_list = create_data(['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'JNJ', 'MA', 'UNH', 'PG', 'HD', 'PYPL', 'BAC', 'DIS', 'VZ', 'XOM', 'NFLX', 'ADBE', 'CSCO', 'T', 'CRM', 'WMT', 'INTC', 'CVX', 'PFE', 'KO', 'CMCSA', 'ABBV', 'ABT', 'MDT', 'ORCL', 'NKE', 'ACN', 'PEP', 'QCOM', 'MRK', 'MMM', 'BMY', 'C', 'MA', 'AXP', 'BA', 'GS', 'IBM', 'MS', 'GE'])        
df_list = create_data(["AAPL", "BAC", "AMZN", "AA", "AXP", "T", "MO", "ABT", "AMAT", "AMGN"])        


In [2]:

# start_date = '2022-01-01'
# end_date = '2022-12-31'


# temp_rp = df_list[0][['Date','Close','RP_MA5_CRP_trend_ma5_backtrack6_alpha10_theta3']]
# temp_tp = df_list[0][['Date','Close','TP_MA5_CTP_trend_ma5_backtrack6_alpha10_theta3']]
# temp_rp.set_index('Date', inplace=True)
# temp_tp.set_index('Date', inplace=True)
# temp_rp = temp_rp[(temp_rp.index >= start_date) & (temp_rp.index <= end_date)]
# temp_tp = temp_tp[(temp_tp.index >= start_date) & (temp_tp.index <= end_date)]

# fig = plt.figure(figsize=(12, 15))  

# # First plot
# ax1 = fig.add_subplot(211)
# ax1.plot(temp_rp.index, temp_rp['Close'])
# buy_signals_rp = temp_rp[temp_rp['RP_MA5_CRP_trend_ma5_backtrack6_alpha10_theta3'] == 1].index
# sell_signals_rp = temp_rp[temp_rp['RP_MA5_CRP_trend_ma5_backtrack6_alpha10_theta3'] == 2].index
# ax1.plot(buy_signals_rp, temp_rp.loc[buy_signals_rp, 'Close'], marker='o', markersize=5, color='g', linestyle='None', label='Buy')
# ax1.plot(sell_signals_rp, temp_rp.loc[sell_signals_rp, 'Close'], marker='o', markersize=5, color='r', linestyle='None', label='Sell')
# ax1.legend()
# ax1.set_title('RP')

# # Second plot
# ax2 = fig.add_subplot(212, sharex=ax1)
# ax2.plot(temp_tp.index, temp_tp['Close'])
# buy_signals_tp = temp_tp[temp_tp['TP_MA5_CTP_trend_ma5_backtrack6_alpha10_theta3'] == 1].index
# sell_signals_tp = temp_tp[temp_tp['TP_MA5_CTP_trend_ma5_backtrack6_alpha10_theta3'] == 2].index
# ax2.plot(buy_signals_tp, temp_tp.loc[buy_signals_tp, 'Close'], marker='o', markersize=5, color='g', linestyle='None', label='Buy')
# ax2.plot(sell_signals_tp, temp_tp.loc[sell_signals_tp, 'Close'], marker='o', markersize=5, color='r', linestyle='None', label='Sell')
# ax2.legend()
# ax2.set_title('TP')

# plt.tight_layout()  
# plt.show()


In [16]:
output_col = 'RP_MA5_CRP_trend_ma5_backtrack6_alpha10_theta3'
X, y = create_sliding_window_data(df_list, 10, output_col)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

y_train_sell_vs_notsell  = np.where(y_train == 2, 1, 0)
y_train_buy_vs_notbuy = np.where(y_train == 1, 1, 0)
y_val_sell_vs_notsell  = np.where(y_val == 2, 1, 0)
y_val_buy_vs_notbuy = np.where(y_val == 1, 1, 0)

# model_drp= Sequential()
# model_drp.add(layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
# model_drp.add(layers.MaxPooling1D(pool_size=2))
# model_drp.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
# model_drp.add(layers.MaxPooling1D(pool_size=2))
# model_drp.add(layers.Flatten())
# model_drp.add(layers.Dense(128, activation='relu'))
# model_drp.add(layers.Dense(1, activation='sigmoid'))
# model_drp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model_drp.fit(X_train, y_train_sell_vs_notsell, validation_data=(X_val, y_val_sell_vs_notsell), epochs=50, batch_size=64, class_weight=class_weights_dict_drp)


class_weights_drp = class_weight.compute_class_weight('balanced', classes=[0, 1], y=y_train_sell_vs_notsell.flatten())
class_weights_dict_drp = {class_index: weight for class_index, weight in zip([0, 1], class_weights_drp)}
model_drp = Sequential()
model_drp.add(LSTM(units=200, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=regularizers.l1(0.01)))
model_drp.add(Dense(units=1, activation='sigmoid'))
model_drp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_drp.fit(X_train, y_train_sell_vs_notsell, validation_data=(X_val, y_val_sell_vs_notsell), epochs=200, batch_size=64, class_weight=class_weights_dict_drp)
save_model(model_drp, 'model_drp.h5')

class_weights_urp = class_weight.compute_class_weight('balanced', classes=[0, 1], y=y_train_buy_vs_notbuy.flatten())
class_weights_dict_urp = {class_index: weight for class_index, weight in zip([0, 1], class_weights_urp)}
model_urp = Sequential()
model_urp.add(LSTM(units=200, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=regularizers.l1(0.01)))
model_urp.add(Dense(units=1, activation='sigmoid'))
model_urp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_urp.fit(X_train, y_train_buy_vs_notbuy, validation_data=(X_val, y_val_buy_vs_notbuy), epochs=200, batch_size=64, class_weight=class_weights_dict_urp)
save_model(model_urp, 'model_urp.h5')




y_pred = model_drp.predict(X_val)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_val_sell_vs_notsell, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)


y_pred = model_urp.predict(X_val)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_val_buy_vs_notbuy, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)


Epoch 1/200
805/805 [==============================] - 10s 11ms/step - loss: 1.1282 - accuracy: 0.8070 - val_loss: 0.4420 - val_accuracy: 0.9297
Epoch 2/200
805/805 [==============================] - 8s 9ms/step - loss: 0.3977 - accuracy: 0.9024 - val_loss: 0.3593 - val_accuracy: 0.9297
Epoch 3/200
805/805 [==============================] - 8s 10ms/step - loss: 0.3894 - accuracy: 0.9277 - val_loss: 0.3717 - val_accuracy: 0.9306
Epoch 4/200
805/805 [==============================] - 8s 10ms/step - loss: 0.2748 - accuracy: 0.9294 - val_loss: 0.3055 - val_accuracy: 0.9298
Epoch 5/200
805/805 [==============================] - 9s 11ms/step - loss: 0.2857 - accuracy: 0.9257 - val_loss: 0.4113 - val_accuracy: 0.9293
Epoch 6/200
805/805 [==============================] - 8s 10ms/step - loss: 0.2860 - accuracy: 0.9290 - val_loss: 0.2490 - val_accuracy: 0.9314
Epoch 7/200
805/805 [==============================] - 9s 11ms/step - loss: 0.2552 - accuracy: 0.9291 - val_loss: 0.2793 - val_accuracy:

805/805 [==============================] - 10s 12ms/step - loss: 0.2034 - accuracy: 0.9295 - val_loss: 0.2708 - val_accuracy: 0.9296
Epoch 58/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2063 - accuracy: 0.9298 - val_loss: 0.2472 - val_accuracy: 0.9301
Epoch 59/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2590 - accuracy: 0.9259 - val_loss: 0.2685 - val_accuracy: 0.9290
Epoch 60/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2021 - accuracy: 0.9298 - val_loss: 0.2685 - val_accuracy: 0.9285
Epoch 61/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1924 - accuracy: 0.9302 - val_loss: 0.2247 - val_accuracy: 0.9299
Epoch 62/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1962 - accuracy: 0.9303 - val_loss: 0.2428 - val_accuracy: 0.9300
Epoch 63/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1953 - accuracy: 0.9306 - val_loss: 0.2677 - val_accuracy

805/805 [==============================] - 10s 12ms/step - loss: 0.2428 - accuracy: 0.9298 - val_loss: 0.2605 - val_accuracy: 0.9288
Epoch 114/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1930 - accuracy: 0.9311 - val_loss: 0.2609 - val_accuracy: 0.9301
Epoch 115/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1895 - accuracy: 0.9315 - val_loss: 0.2639 - val_accuracy: 0.9306
Epoch 116/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2131 - accuracy: 0.9307 - val_loss: 0.2447 - val_accuracy: 0.9308
Epoch 117/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2130 - accuracy: 0.9308 - val_loss: 0.3895 - val_accuracy: 0.9269
Epoch 118/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2138 - accuracy: 0.9297 - val_loss: 0.2342 - val_accuracy: 0.9300
Epoch 119/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1877 - accuracy: 0.9319 - val_loss: 0.2463 - val_ac

805/805 [==============================] - 9s 11ms/step - loss: 0.1973 - accuracy: 0.9334 - val_loss: 0.2993 - val_accuracy: 0.9300
Epoch 170/200
805/805 [==============================] - 9s 12ms/step - loss: 0.2774 - accuracy: 0.9297 - val_loss: 0.2642 - val_accuracy: 0.9302
Epoch 171/200
805/805 [==============================] - 9s 11ms/step - loss: 0.1861 - accuracy: 0.9314 - val_loss: 0.2302 - val_accuracy: 0.9306
Epoch 172/200
805/805 [==============================] - 9s 12ms/step - loss: 0.2206 - accuracy: 0.9327 - val_loss: 0.3184 - val_accuracy: 0.9307
Epoch 173/200
805/805 [==============================] - 9s 12ms/step - loss: 0.2469 - accuracy: 0.9300 - val_loss: 0.2528 - val_accuracy: 0.9306
Epoch 174/200
805/805 [==============================] - 9s 12ms/step - loss: 0.1833 - accuracy: 0.9331 - val_loss: 0.2374 - val_accuracy: 0.9315
Epoch 175/200
805/805 [==============================] - 9s 12ms/step - loss: 0.2069 - accuracy: 0.9334 - val_loss: 0.3237 - val_accuracy:

805/805 [==============================] - 10s 12ms/step - loss: 0.1630 - accuracy: 0.9561 - val_loss: 0.1979 - val_accuracy: 0.9564
Epoch 26/200
805/805 [==============================] - 10s 13ms/step - loss: 0.1650 - accuracy: 0.9567 - val_loss: 0.2094 - val_accuracy: 0.9563
Epoch 27/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2028 - accuracy: 0.9542 - val_loss: 0.2342 - val_accuracy: 0.9531
Epoch 28/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1555 - accuracy: 0.9570 - val_loss: 0.1884 - val_accuracy: 0.9562
Epoch 29/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1538 - accuracy: 0.9567 - val_loss: 0.1979 - val_accuracy: 0.9562
Epoch 30/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2010 - accuracy: 0.9548 - val_loss: 0.1816 - val_accuracy: 0.9562
Epoch 31/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1511 - accuracy: 0.9569 - val_loss: 0.2359 - val_accuracy

805/805 [==============================] - 10s 13ms/step - loss: 0.1478 - accuracy: 0.9572 - val_loss: 0.1804 - val_accuracy: 0.9562
Epoch 82/200
805/805 [==============================] - 10s 13ms/step - loss: 0.1475 - accuracy: 0.9573 - val_loss: 0.1853 - val_accuracy: 0.9569
Epoch 83/200
805/805 [==============================] - 10s 13ms/step - loss: 0.1805 - accuracy: 0.9541 - val_loss: 0.2388 - val_accuracy: 0.9549
Epoch 84/200
805/805 [==============================] - 10s 13ms/step - loss: 0.1475 - accuracy: 0.9574 - val_loss: 0.2030 - val_accuracy: 0.9564
Epoch 85/200
805/805 [==============================] - 10s 13ms/step - loss: 0.2128 - accuracy: 0.9547 - val_loss: 0.2934 - val_accuracy: 0.9555
Epoch 86/200
805/805 [==============================] - 10s 13ms/step - loss: 0.1601 - accuracy: 0.9566 - val_loss: 0.1978 - val_accuracy: 0.9564
Epoch 87/200
805/805 [==============================] - 10s 13ms/step - loss: 0.1471 - accuracy: 0.9572 - val_loss: 0.1838 - val_accuracy

Epoch 137/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1543 - accuracy: 0.9573 - val_loss: 0.1820 - val_accuracy: 0.9564
Epoch 138/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1702 - accuracy: 0.9573 - val_loss: 0.1918 - val_accuracy: 0.9570
Epoch 139/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1465 - accuracy: 0.9576 - val_loss: 0.1883 - val_accuracy: 0.9569
Epoch 140/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1697 - accuracy: 0.9569 - val_loss: 0.1762 - val_accuracy: 0.9580
Epoch 141/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1439 - accuracy: 0.9578 - val_loss: 0.1934 - val_accuracy: 0.9574
Epoch 142/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1484 - accuracy: 0.9578 - val_loss: 0.1963 - val_accuracy: 0.9575
Epoch 143/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2043 - accuracy: 0.9547 - val_loss: 0

Epoch 193/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1769 - accuracy: 0.9576 - val_loss: 0.1946 - val_accuracy: 0.9566
Epoch 194/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1481 - accuracy: 0.9579 - val_loss: 0.1954 - val_accuracy: 0.9568
Epoch 195/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1719 - accuracy: 0.9572 - val_loss: 0.1809 - val_accuracy: 0.9566
Epoch 196/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1445 - accuracy: 0.9580 - val_loss: 0.1967 - val_accuracy: 0.9566
Epoch 197/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1944 - accuracy: 0.9569 - val_loss: 0.3527 - val_accuracy: 0.9546
Epoch 198/200
805/805 [==============================] - 10s 12ms/step - loss: 0.2077 - accuracy: 0.9560 - val_loss: 0.2037 - val_accuracy: 0.9566
Epoch 199/200
805/805 [==============================] - 10s 12ms/step - loss: 0.1443 - accuracy: 0.9579 - val_loss: 0

In [4]:
output_col = 'TP_MA5_CTP_trend_ma5_backtrack6_alpha10_theta3'

X, y = create_sliding_window_data(df_list, 10, output_col)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

y_train_sell_vs_notsell  = np.where(y_train == 2, 1, 0)
y_train_buy_vs_notbuy = np.where(y_train == 1, 1, 0)
y_val_sell_vs_notsell  = np.where(y_val == 2, 1, 0)
y_val_buy_vs_notbuy = np.where(y_val == 1, 1, 0)



class_weights_dtp = class_weight.compute_class_weight('balanced', classes=[0, 1], y=y_train_sell_vs_notsell.flatten())
class_weights_dict_dtp = {class_index: weight for class_index, weight in zip([0, 1], class_weights_dtp)}
model_dtp = Sequential()
model_dtp.add(LSTM(units=200, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=regularizers.l1(0.01)))
model_dtp.add(Dense(units=1, activation='sigmoid'))
model_dtp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dtp.fit(X_train, y_train_sell_vs_notsell, validation_data=(X_val, y_val_sell_vs_notsell), epochs=50, batch_size=64, class_weight=class_weights_dict_dtp)
save_model(model_dtp, 'model_dtp.h5')

class_weights_utp = class_weight.compute_class_weight('balanced', classes=[0, 1], y=y_train_buy_vs_notbuy.flatten())
class_weights_dict_utp = {class_index: weight for class_index, weight in zip([0, 1], class_weights_utp)}
model_utp = Sequential()
model_utp.add(LSTM(units=200, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=regularizers.l1(0.01)))
model_utp.add(Dense(units=1, activation='sigmoid'))
model_utp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_utp.fit(X_train, y_train_buy_vs_notbuy, validation_data=(X_val, y_val_buy_vs_notbuy), epochs=50, batch_size=64, class_weight=class_weights_dict_utp)
save_model(model_utp, 'model_utp.h5')



Epoch 1/50
805/805 [==============================] - 12s 12ms/step - loss: 1.0589 - accuracy: 0.7871 - val_loss: 0.3665 - val_accuracy: 0.9015
Epoch 2/50
805/805 [==============================] - 9s 11ms/step - loss: 0.3478 - accuracy: 0.9019 - val_loss: 0.3809 - val_accuracy: 0.9117
Epoch 3/50
805/805 [==============================] - 9s 11ms/step - loss: 0.2906 - accuracy: 0.9242 - val_loss: 0.2962 - val_accuracy: 0.9319
Epoch 4/50
805/805 [==============================] - 9s 12ms/step - loss: 0.2778 - accuracy: 0.9255 - val_loss: 0.2346 - val_accuracy: 0.9473
Epoch 5/50
805/805 [==============================] - 9s 12ms/step - loss: 0.2682 - accuracy: 0.9292 - val_loss: 0.3044 - val_accuracy: 0.9299
Epoch 6/50
805/805 [==============================] - 9s 11ms/step - loss: 0.2835 - accuracy: 0.9266 - val_loss: 0.2477 - val_accuracy: 0.9416
Epoch 7/50
805/805 [==============================] - 10s 12ms/step - loss: 0.2574 - accuracy: 0.9302 - val_loss: 0.2866 - val_accuracy: 0.93

Epoch 8/50
805/805 [==============================] - 8s 10ms/step - loss: 0.2879 - accuracy: 0.9082 - val_loss: 0.2920 - val_accuracy: 0.9213
Epoch 9/50
805/805 [==============================] - 8s 10ms/step - loss: 0.2900 - accuracy: 0.9087 - val_loss: 0.3358 - val_accuracy: 0.9093
Epoch 10/50
805/805 [==============================] - 8s 10ms/step - loss: 0.2920 - accuracy: 0.9106 - val_loss: 0.3172 - val_accuracy: 0.9156
Epoch 11/50
805/805 [==============================] - 8s 10ms/step - loss: 0.2814 - accuracy: 0.9110 - val_loss: 0.3435 - val_accuracy: 0.9028
Epoch 12/50
805/805 [==============================] - 8s 10ms/step - loss: 0.2834 - accuracy: 0.9107 - val_loss: 0.2790 - val_accuracy: 0.9220
Epoch 13/50
805/805 [==============================] - 8s 10ms/step - loss: 0.2743 - accuracy: 0.9123 - val_loss: 0.3160 - val_accuracy: 0.9171
Epoch 14/50
805/805 [==============================] - 8s 10ms/step - loss: 0.2789 - accuracy: 0.9118 - val_loss: 0.2822 - val_accuracy: 0

# OOS

In [5]:
model_urp = load_model('model_urp.h5')
model_drp = load_model('model_drp.h5')
model_utp = load_model('model_utp.h5')
model_dtp = load_model('model_dtp.h5')


In [6]:
# df_oos_list = create_data(['300015.SZ','300379.SZ','603881.SS','688981.SS','300058.SZ'])
df_oos_list = create_data(['^GSPC'])
# df_oos_list = create_data(['BTC-USD'])



In [7]:
output_col = 'RP_MA5_CRP_trend_ma5_backtrack6_alpha10_theta3'
X_oos, y_oos = create_sliding_window_data(df_oos_list, 10, output_col)

y_oos_sell_vs_notsell = np.where(y_oos == 2, 1, 0)
y_oos_buy_vs_notbuy = np.where(y_oos == 1, 1, 0)


y_pred = model_drp.predict(X_oos)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_oos_sell_vs_notsell, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

y_pred = model_urp.predict(X_oos)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_oos_buy_vs_notbuy, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

X_oos_balanced, y_oos_sell_vs_notsell_balanced = re_balance(X_oos, y_oos_sell_vs_notsell)
y_pred = model_drp.predict(X_oos_balanced)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_oos_sell_vs_notsell_balanced, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

X_oos_balanced, y_oos_buy_vs_notbuy_balanced = re_balance(X_oos, y_oos_buy_vs_notbuy)
y_pred = model_urp.predict(X_oos_balanced)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_oos_buy_vs_notbuy_balanced, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)




202/202 [==============================] - 1s 4ms/step
Precision: [1.         0.08777429]
Recall: [0.90886314 1.        ]
F1-score: [0.95225595 0.16138329]
202/202 [==============================] - 1s 4ms/step
Precision: [1.        0.1779661]
Recall: [0.95438156 1.        ]
F1-score: [0.97665838 0.30215827]
4/4 [==============================] - 0s 5ms/step
Precision: [1.         0.91803279]
Recall: [0.91071429 1.        ]
F1-score: [0.95327103 0.95726496]
4/4 [==============================] - 0s 4ms/step
Precision: [1.       0.984375]
Recall: [0.98412698 1.        ]
F1-score: [0.992      0.99212598]


In [9]:
output_col = 'TP_MA5_CTP_trend_ma5_backtrack6_alpha10_theta3'
X_oos, y_oos = create_sliding_window_data(df_oos_list, 10, output_col)

y_oos_sell_vs_notsell = np.where(y_oos == 2, 1, 0)
y_oos_buy_vs_notbuy = np.where(y_oos == 1, 1, 0)
y_pred = model_dtp.predict(X_oos)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_oos_sell_vs_notsell, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

y_pred = model_utp.predict(X_oos)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_oos_buy_vs_notbuy, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

X_oos_balanced, y_oos_sell_vs_notsell_balanced = re_balance(X_oos, y_oos_sell_vs_notsell)
y_pred = model_dtp.predict(X_oos_balanced)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_oos_sell_vs_notsell_balanced, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

X_oos_balanced, y_oos_buy_vs_notbuy_balanced = re_balance(X_oos, y_oos_buy_vs_notbuy)
y_pred = model_utp.predict(X_oos_balanced)
y_pred = np.round(y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_oos_buy_vs_notbuy_balanced, y_pred, average=None)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)


202/202 [==============================] - 1s 3ms/step
Precision: [0.99949023 0.26929982]
Recall: [0.93528383 0.98039216]
F1-score: [0.96632167 0.42253521]
202/202 [==============================] - 1s 3ms/step
Precision: [0.99962853 0.11153119]
Recall: [0.85131288 0.98333333]
F1-score: [0.91952845 0.20033956]
10/10 [==============================] - 0s 3ms/step
Precision: [0.97902098 0.9202454 ]
Recall: [0.91503268 0.98039216]
F1-score: [0.94594595 0.94936709]
8/8 [==============================] - 0s 3ms/step
Precision: [0.98058252 0.86131387]
Recall: [0.84166667 0.98333333]
F1-score: [0.9058296  0.91828794]


# BACKTEST

In [ ]:
model_urp = load_model('model_urp.h5')
model_drp = load_model('model_drp.h5')
model_utp = load_model('model_utp.h5')
model_dtp = load_model('model_dtp.h5')


In [ ]:

# def strategy(df, start_date, end_date):
#     df = df[(df.index >= start_date) & (df.index <= end_date)]
#     df = df.reset_index()

#     shares = 0
#     stock_value = 0
#     cash_value = 1000000
#     purchase_price = None
#     holding_days = 0
#     last_sell_price = 0

#     for i, row in df.iterrows():
#         if holding_days > 10 or (purchase_price is not None and (row['Open'] - purchase_price) / purchase_price < -0.03):
#             cash_value += shares * row['Open']
#             shares = 0
#             purchase_price = None
#             holding_days = 0

#         elif i > 0 and df.loc[i-1, 'prediction_decision'] == 'buy':
#             shares_to_buy = np.floor(cash_value / row['Open'])
#             shares += shares_to_buy
#             cash_value -= shares_to_buy * row['Open']
#             purchase_price = row['Open']
#             holding_days = 0

#         elif i > 0 and df.loc[i-1, 'prediction_decision'] == 'sell':
#             cash_value += shares * row['Open']
#             shares = 0
#             purchase_price = None
#             holding_days = 0
#             last_sell_price = row['Open']

#         else:
#             holding_days += 1

#         stock_value = shares * row['Close']
#         df.loc[i, 'Shares'] = shares
#         df.loc[i, 'Stock Value'] = stock_value
#         df.loc[i, 'Cash Value'] = cash_value

#         df['total_value'] = df['Stock Value'] + df['Cash Value']
#         metric = (df.iloc[-1]['total_value'] - df.iloc[0]['total_value']) / df.iloc[0]['total_value']

#     return df, metric


def strategy(df, start_date, end_date):
    df = df[(df.index >= start_date) & (df.index <= end_date)]
    df = df.reset_index()

    shares = 0
    stock_value = 0
    cash_value = 1000000
    purchase_price = None
    holding_days = 0
    last_sell_price = 0

    for i, row in df.iterrows():
        if holding_days > 10:
            cash_value += shares * row['Open']
            shares = 0
            purchase_price = None
            holding_days = 0

        elif i > 0 and df.loc[i-1, 'prediction_decision'] == 'buy':
            shares_to_buy = np.floor(cash_value / row['Open'])
            shares += shares_to_buy
            cash_value -= shares_to_buy * row['Open']
            purchase_price = row['Open']
            holding_days = 0

        elif i > 0 and df.loc[i-1, 'prediction_decision'] == 'sell':
            cash_value += shares * row['Open']
            shares = 0
            purchase_price = None
            holding_days = 0
            last_sell_price = row['Open']

        else:
            holding_days += 1

        stock_value = shares * row['Close']
        df.loc[i, 'Shares'] = shares
        df.loc[i, 'Stock Value'] = stock_value
        df.loc[i, 'Cash Value'] = cash_value

        df['total_value'] = df['Stock Value'] + df['Cash Value']
        metric = (df.iloc[-1]['total_value'] - df.iloc[0]['total_value']) / df.iloc[0]['total_value']

    return df, metric

# def strategy(df, start_date, end_date):
#     df = df[(df.index >= start_date) & (df.index <= end_date)]
#     df = df.reset_index()

#     shares = 0
#     stock_value = 0
#     cash_value = 1000000
#     purchase_price = None
#     holding_days = 0
#     last_sell_price = 0
#     next_buy_index = None

#     for i, row in df.iterrows():
#         if holding_days > 10 or (next_buy_index is not None and i >= next_buy_index):
#             cash_value += shares * row['Open']
#             shares = 0
#             purchase_price = None
#             holding_days = 0
#             next_buy_index = None

#         elif i > 0 and df.loc[i-1, 'prediction_decision'] == 'buy':
#             shares_to_buy = np.floor(cash_value / row['Open'])
#             shares += shares_to_buy
#             cash_value -= shares_to_buy * row['Open']
#             purchase_price = row['Open']
#             holding_days = 0
#             next_buy_index = i + 10 if i + 10 < len(df) else len(df) - 1

#         else:
#             holding_days += 1

#         stock_value = shares * row['Close']
#         df.loc[i, 'Shares'] = shares
#         df.loc[i, 'Stock Value'] = stock_value
#         df.loc[i, 'Cash Value'] = cash_value

#         df['total_value'] = df['Stock Value'] + df['Cash Value']
#         metric = (df.iloc[-1]['total_value'] - df.iloc[0]['total_value']) / df.iloc[0]['total_value']

#     return df, metric

# def strategy(df, start_date, end_date):
#     df = df[(df.index >= start_date) & (df.index <= end_date)]
#     df = df.reset_index()

#     shares = 0
#     stock_value = 0
#     cash_value = 1000000

#     for i, row in df.iterrows():
#         if i > 0 and df.loc[i-1, 'prediction_decision'] == 'sell':
#             cash_value += shares * row['Open']
#             shares = 0

#         elif i > 0 and df.loc[i-1, 'prediction_decision'] == 'buy':
#             shares_to_buy = np.floor(cash_value / row['Open'])
#             shares += shares_to_buy
#             cash_value -= shares_to_buy * row['Open']

#         stock_value = shares * row['Close']
#         df.loc[i, 'Shares'] = shares
#         df.loc[i, 'Stock Value'] = stock_value
#         df.loc[i, 'Cash Value'] = cash_value

#         df['total_value'] = df['Stock Value'] + df['Cash Value']
#         metric = (df.iloc[-1]['total_value'] - df.iloc[0]['total_value']) / df.iloc[0]['total_value']

#     return df, metric





def baseline(df, start_date, end_date):
    df = df[(df.index >= start_date) & (df.index <= end_date)]
    df = df.reset_index()
    start_price = df.iloc[0]['Close']
    end_price = df.iloc[-1]['Close']
    metric = (end_price - start_price) / start_price
    return metric

    
def apply_strategy(row):
    _, metric = strategy(df, row['min'], row['max'])
    return metric



def apply_baseline(row):
    metric = baseline(df, row['min'], row['max'])
    return metric

In [ ]:


# df = create_data(['BTCC-B.TO'], False)[0]
# df = create_data(['NVDA'], False)[0]
# df = create_data(['600019.SS'], False)[0]

# df = create_data(['BTC-USD'], False)[0]
df = create_data(['^GSPC'], False)[0]

df=df[df['Date']>='1990']
window_size=10
X_data = df[input_cols].values
X = []
scaler = StandardScaler()

for i in range(len(X_data)+1):
    if i < window_size:
        pass
    else:
        X_data_scaled = scaler.fit_transform(X_data[i-window_size:i])
        X.append(X_data_scaled)
X = np.array(X)

prediction_drp = model_drp.predict(X)
prediction_urp = model_urp.predict(X)
prediction_dtp = model_dtp.predict(X)
prediction_utp = model_utp.predict(X)
prediction_drp_padded = np.concatenate([np.full((window_size-1, prediction_drp.shape[1]), np.nan), prediction_drp], axis=0)
prediction_urp_padded = np.concatenate([np.full((window_size-1, prediction_urp.shape[1]), np.nan), prediction_urp], axis=0)
prediction_dtp_padded = np.concatenate([np.full((window_size-1, prediction_dtp.shape[1]), np.nan), prediction_dtp], axis=0)
prediction_utp_padded = np.concatenate([np.full((window_size-1, prediction_utp.shape[1]), np.nan), prediction_utp], axis=0)

df['predict_drp'] = prediction_drp_padded
df['predict_urp'] = prediction_urp_padded
df['predict_dtp'] = prediction_dtp_padded
df['predict_utp'] = prediction_utp_padded


df['prediction_rp'] = np.where(df['predict_drp'] > 0.5, 'sell', np.where(df['predict_urp'] > 0.5, 'buy', np.nan))
df['prediction_tp'] = np.where(df['predict_dtp'] > 0.5, 'sell', np.where(df['predict_utp'] > 0.5, 'buy', np.nan))

# df['prediction_decision'] = np.where(df['prediction_rp'].isin(['sell', 'buy']), df['prediction_rp'], df['prediction_tp'])
# df['prediction_decision'] = np.where(df['prediction_rp'] == 'sell', 'sell', df['prediction_tp'])
df['prediction_decision'] = df['prediction_tp']

df.set_index('Date', inplace=True)


In [ ]:
df_backtest = df.copy()
df_backtest = df_backtest.reset_index()
df_backtest['Year'] = df_backtest['Date'].dt.year
df_backtest = df_backtest.groupby('Year')['Date'].agg(['min', 'max'])
df_backtest['annual_return'] = df_backtest.apply(apply_strategy, axis=1)
df_backtest['baseline'] = df_backtest.apply(apply_baseline, axis=1)
df_backtest['delta_return'] = df_backtest['annual_return'] - df_backtest['baseline']
print(df_backtest)
print(f"strategy return {df_backtest['annual_return'].mean()}, baseline return {df_backtest['baseline'].mean()}")
print(f"strategy risk {df_backtest['annual_return'].std()}, baseline risk {df_backtest['baseline'].std()}")
print(f"strategy risk adjusted return {df_backtest['annual_return'].mean()/df_backtest['annual_return'].std()}, baseline risk adjusted return {df_backtest['baseline'].mean()/df_backtest['baseline'].std()}")


In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'
df_subset,metric_subset = strategy(df, start_date, end_date)
df_subset.set_index('Date', inplace=True)
print(metric_subset)


In [ ]:
fig = plt.figure(figsize=(12, 15))  

# First plot
ax1 = fig.add_subplot(511)
ax1.plot(df_subset.index, df_subset['Close'], color='blue')
ax1.set_ylabel('Close Price')
ax1.set_title('Date vs Close Price')

# Second plot
ax2 = fig.add_subplot(512, sharex=ax1)
ax2.plot(df_subset.index, df_subset['total_value'], color='red')
ax2.set_ylabel('Total Value', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_title('Date vs Total Value')

# Third plot
ax3 = fig.add_subplot(513, sharex=ax1)
ax3.plot(df_subset.index, df_subset['Close'])
buy_signals = df_subset[df_subset['prediction_tp'] == 'buy'].index
sell_signals = df_subset[df_subset['prediction_tp'] == 'sell'].index
ax3.plot(buy_signals, df_subset.loc[buy_signals, 'Close'], marker='o', markersize=5, color='g', linestyle='None', label='Buy')
ax3.plot(sell_signals, df_subset.loc[sell_signals, 'Close'], marker='o', markersize=5, color='r', linestyle='None', label='Sell')
ax3.legend()
ax3.set_title('TP')

# Fourth plot
ax4 = fig.add_subplot(514, sharex=ax1)
ax4.plot(df_subset.index, df_subset['Close'])
buy_signals = df_subset[df_subset['prediction_rp'] == 'buy'].index
sell_signals = df_subset[df_subset['prediction_rp'] == 'sell'].index
ax4.plot(buy_signals, df_subset.loc[buy_signals, 'Close'], marker='o', markersize=5, color='g', linestyle='None', label='Buy')
ax4.plot(sell_signals, df_subset.loc[sell_signals, 'Close'], marker='o', markersize=5, color='r', linestyle='None', label='Sell')
ax4.legend()
ax4.set_title('RP')

# Fifth plot
ax5 = fig.add_subplot(515, sharex=ax1)
ax5.plot(df_subset.index, df_subset['Close'])
buy_signals = df_subset[df_subset['prediction_decision'] == 'buy'].index
sell_signals = df_subset[df_subset['prediction_decision'] == 'sell'].index
ax5.plot(buy_signals, df_subset.loc[buy_signals, 'Close'], marker='o', markersize=5, color='g', linestyle='None', label='Buy')
ax5.plot(sell_signals, df_subset.loc[sell_signals, 'Close'], marker='o', markersize=5, color='r', linestyle='None', label='Sell')
ax5.legend()
ax5.set_title('Decision')




plt.tight_layout()  
plt.show()
